In [1]:
import pandas as pd
import itertools

In [2]:
def get_values(df, mapping, prefix='personal', skip_on=None):
    ags = None
    for i, row in df.iterrows():
        if not pd.isnull(row['key']) and row['key'].startswith('01 '):
            ags = row['key'].replace(' ', '')
            continue
        if ags is None:
            continue
        if pd.isnull(row['key']):
            continue
        if skip_on is not None and row['key'] in skip_on:
            ags = None
            continue
        for k, v in mapping.items():
            if k in row['key']:
                if isinstance(row['value'], str):
                    val = int(row['value'].replace('-', '0').strip())
                else:
                    val = row['value']
                yield ags, prefix, v, val

In [3]:
PATH = 'raw/Pflegestatistik_2013_Schleswig-Holstein.xls'
generators = []

In [4]:
personal_mapping = {
    'Vollzeitbeschäftigt': 'fully_employed',
    'Teilzeitbeschäftigt': 'part-time',
    'Pflege und Betreuung': 'nursing_and_care',
    'soziale Betreuung': 'social_care',
    'zusätzliche Betreuung (§ 87b SGB XI)': 'other_care'
}

df = pd.read_excel(PATH, sheetname='Personal', skiprows=8)
df = df.rename(columns={
    'Unnamed: 0': 'key',
    'Insgesamt': 'value'
})
generators.append(get_values(df, personal_mapping))
df.head()

,key,value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Personal insgesamt,29707,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,"Auszubildende/-r, (Um-)Schüler/-in",1603,NaN,NaN,NaN,NaN
3,Überwiegender Tätigkeitsbereich im Pflegeheim,NaN,NaN,NaN,NaN,NaN
4,Pflege und Betreuung,18870,NaN,NaN,NaN,NaN


In [5]:
cost_mapping = {
    'Pflegeklasse 1': 'nursing_class_1',
    'Pflegeklasse 2': 'nursing_class_2',
    'Pflegeklasse 3': 'nursing_class_3',
    'Verpflegung': 'food',
}

df = pd.read_excel(PATH, sheetname='Vergütung', skiprows=14)
df = df.rename(columns={
    'Unnamed: 0': 'key',
    'Insgesamt                               ': 'value'
})

generators.append(get_values(df, cost_mapping, prefix='costs', skip_on=['Vergütung für Kurzzeitpflege  ']))
df.head()

,key,value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Vergütung für vollstationäre,NaN,NaN,NaN,NaN,NaN,NaN
1,Dauerpflege,NaN,NaN,NaN,NaN,NaN,NaN
2,Pflegesatz,NaN,NaN,NaN,NaN,NaN,NaN
3,Pflegeklasse 1,45.09,NaN,NaN,NaN,NaN,NaN
4,Pflegeklasse 2,56.74,NaN,NaN,NaN,NaN,NaN


In [6]:
bed_mapping = {
  '1-Bett-Zimmern': '1-bed',
  '2-Bett-Zimmern': '2-bed',
  '3-Bett-Zimmern': '3-bed',
  '4 und mehr-Bett-Zimmern': '4-bed'
}

df = pd.read_excel(PATH, sheetname='verfügbare Plätze Betten', skiprows=13)
df = df.rename(columns={
    'Unnamed: 0': 'key',
    'Insgesamt                               ': 'value'
})

generators.append(get_values(df, bed_mapping, prefix='beds'))
df.head()

,key,value
0,Anzahl ...,NaN
1,Verfügbare Plätze insgesamt,40703
2,NaN,NaN
3,vollstationäre Pflege,NaN
4,zusammen,39508


In [7]:
recipient_mapping = {
    'Pflegestufe I ': 'nursing_class_1',
    'Pflegestufe II ': 'nursing_class_2',
    'Pflegestufe III ': 'nursing_class_3',
    'Pflegestufe zugeordnet': 'nursing_class_unknown',
}

df = pd.read_excel(PATH, sheetname='Pflegebedürftige', skiprows=10)
df = df.rename(columns={
    'Unnamed: 0': 'key',
    'Unnamed: 2': 'value'
})

generators.append(get_values(df, recipient_mapping, prefix='recipients'))
df.head()

,key,Insgesamt,value
0,Insgesamt,35248,33483
1,Pflegestufe I,14621,13819
2,Pflegestufe II,13147,12425
3,Pflegestufe III,6150,5978
4,Bisher noch keiner,NaN,NaN


In [8]:
df = pd.DataFrame(list(itertools.chain(*generators)))
df = df.rename(columns={
    0: 'ags',
    1: 'type',
    2: 'key',
    3: 'value'
})
df['state'] = 'Schleswig-Holstein'
df.to_csv('schleswig-holstein.csv')
df.head()

,ags,type,key,value,state
0,01001,personal,nursing_and_care,497.0,Schleswig-Holstein
1,01001,personal,social_care,36.0,Schleswig-Holstein
2,01001,personal,other_care,31.0,Schleswig-Holstein
3,01001,personal,fully_employed,218.0,Schleswig-Holstein
4,01001,personal,part-time,475.0,Schleswig-Holstein
